### Importing Required Depeendencies

In [1]:
from datetime import datetime
from collections import namedtuple

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

### Starting Spark Session

In [2]:
spark = SparkSession.builder \
        .master("spark://ny-trips-vm.europe-west1-b.c.ny-trips-de.internal:7077") \
        .appName('test') \
        .getOrCreate()

22/06/09 10:11:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/09 10:11:51 WARN SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [19]:
spark

In [9]:
df_green = spark.read.parquet("data/raw/green/*/*")

In [10]:
df_yellow = spark.read.parquet("data/raw/yellow/*/*")

### Combining Yellow and Green Taxi Data

In [11]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [12]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [13]:
common_columns = ['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [14]:
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [15]:
df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [16]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [50]:
# Register Dataframe as a Table
df_trips_data.registerTempTable('trips_data')

NameError: name 'df_trips_data' is not defined

In [79]:
df_result = spark.sql(
"""
SELECT
  -- Revenue Grouping
  PULocationID AS revenue_zone,
  date_trunc('month', pickup_datetime) AS revenue_month,
  service_type,

  -- Revenue Calculation
  SUM(fare_amount) AS revenue_monthly_fare,
  SUM(extra) AS revenue_monthly_extra,
  SUM(mta_tax) AS revenue_monthly_mta_tax,
  SUM(tip_amount) AS revenue_monthly_tip_amount,
  SUM(tolls_amount) AS revenue_monthly_tolls_amount,
  SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
  SUM(total_amount) AS revenue_monthly_total_amount,
  SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

  -- Aggregated Counts
  AVG(passenger_count) AS avg_monthly_passenger_count,
  AVG(trip_distance) AS avg_monthly_trip_distance

  FROM tripS_data
  GROUP BY 1, 2, 3
""")

In [82]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')